## load data

In [1]:
import numpy as np
import os
import pickle 
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold

In [ ]:
# load data: 
data = pd.read_csv('1_R/ML_7111genes.csv',index_col=0)
print(data.shape)

gene_names = data.drop(['cohort','label'],axis=1).columns
print(len(gene_names))

In [ ]:
data = data[data['label'].isin(['DN', 'HCC','Y'])]
print(data.shape)
print(data['label'].value_counts())

enc = LabelEncoder()   # LabelEncoder DM,HCC,Y
data['label'] = enc.fit_transform(data['label'])   

inter = data.loc[data['cohort']=='inter']
X_inter = inter.drop(['label','cohort'],axis=1).astype(np.float64)
y_inter = inter['label']

exter = data.loc[data['cohort']=='exter']
X_exterTest = exter.drop(['label','cohort'],axis=1).astype(np.float64)
y_exterTest = exter['label']

print(y_inter.shape, y_exterTest.shape)
print(X_inter.shape, X_exterTest.shape)

# xgboost

1. screen features

In [23]:
import xgboost as xgb
from sklearn.multiclass import OneVsOneClassifier
from collections import Counter
import json

In [ ]:
# record the importance score and then sum.
importance_scores = np.zeros((X_inter.shape[1],))
for seed in range(10):  # 10 times-5 fold
    
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)
    for train_index, test_index in kf.split(X_inter): # 1/5
        X_train = X_inter.iloc[train_index]
        y_train = y_inter.iloc[train_index]
        
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)

        # screen features
        clf = OneVsOneClassifier(xgb.XGBClassifier(random_state=42))
        clf.fit(X_train, y_train)

        for estimator in clf.estimators_:
            importance_scores += estimator.feature_importances_  # importance 

# mean 
average_importance_scores = importance_scores / (10* 5)  # total

# top 10 
top10_idx = np.argsort(average_importance_scores)[-10:][::-1]  # top 10 index
top10_genes = gene_names[top10_idx]  # gene name
top10_scores = average_importance_scores[top10_idx]  # importance score

gene_importance_dict = dict(zip(top10_genes, top10_scores))

with open('2_RNAmodel_vexter/gene_importance-sumscore.json', 'w') as json_file:
    json.dump(gene_importance_dict, json_file)

print("Top 10 :")
for gene, score in zip(top10_genes, top10_scores):
    print(f"{gene}: {score:.4f}")

2. visualization: importance score heatmap

In [22]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
import json

with open('gene_importance-sumscore.json', 'r') as json_file:
    gene_importance_dict = json.load(json_file)

In [ ]:
gene_importance = dict(sorted(gene_importance_dict.items(), key=lambda item: item[1], reverse=True))
top10_genes = list(gene_importance.keys())[:10]

scores = np.array([gene_importance_dict[gene] for gene in top10_genes])
scores = (scores - np.mean(scores)) / np.std(scores)
scores

In [ ]:
plt.figure(figsize=(3, 3))

scores[0] = 1 # max value
top10_score_array = scores.reshape(-1, 1)

plt.imshow(top10_score_array, cmap='OrRd', interpolation='nearest')
plt.colorbar()
plt.xticks([])
plt.yticks(ticks=np.arange(len(top10_genes)), labels=top10_genes, 
        rotation=45, ha='right', fontdict={'fontstyle': 'italic'})

plt.title('Gene Importance Scores')

plt.savefig('importance_score.pdf', bbox_inches='tight')
plt.show()

# xgboost-Hyperparameter

1. Hyperparameter traversal

In [6]:
import json
import pickle
import os
import numpy as np
import itertools

In [ ]:
with open('2_RNAmodel_vexter/geneSumscore/gene_importance-sumscore.json', 'r') as json_file:
    gene_importance = json.load(json_file)

top10_genes = gene_importance.keys()
top10_indices = [list(gene_names).index(top10) for top10 in top10_genes]


X_inter2 = X_inter.iloc[:, top10_indices] 
print(X_inter2.shape)

X_exterTest2 = X_exterTest.iloc[:, top10_indices] 
print(X_exterTest2.shape)

# save
with open(f'2_RNAmodel_vexter/geneSumscore/data_inter-exter.pkl', 'wb') as f:
    pickle.dump((X_inter2, y_inter, X_exterTest2, y_exterTest), f)

In [33]:
with open(f'geneSumscore/data_inter-exter.pkl', 'rb') as f:
    X_inter2, y_inter, X_exterTest2, y_exterTest  = pickle.load(f)

param_grid = {
    'max_depth':range(1,3,1),
    'n_estimators': np.arange(10,50,10),
    'learning_rate': [i/10.0 for i in range(1, 5, 2)],
    
    # 'min_child_weight':[3,4,5],
    # 'gamma':[i/10.0 for i in range(1, 9, 2)],
    # 'subsample':[i/10.0 for i in range(1, 9, 2)],
    # 'colsample_bytree':[i/10.0 for i in range(1, 9, 2)],
}

param_values = [param_grid[param] for param in param_grid]
param_combinations = list(itertools.product(*param_values))

In [59]:
for idx, combination in enumerate(param_combinations): # every hyperparameter combination
    param = dict(zip(param_grid.keys(), combination))
    
    y_pred_all, y_pred_proba_all, y_true_all = [],[],[]
    
    for seed in range(10):  # 10 times - 5 folds
        
        kf = KFold(n_splits=5, shuffle=True, random_state=seed)
        y_pred, y_pred_proba, y_true = [],[],[]

        for train_index, test_index in kf.split(X_inter): # 1/5
            X_train, X_test = X_inter.iloc[train_index], X_inter.iloc[test_index]
            y_train, y_test = y_inter.iloc[train_index], y_inter.iloc[test_index]
            
            scaler = StandardScaler().fit(X_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)

            # modeling
            clf = OneVsOneClassifier(xgb.XGBClassifier(**param))
            clf.fit(X_train, y_train)
            
            # pred
            y_pred.extend(clf.predict(X_test)) 
            y_true.extend(y_test)

        y_pred_all.append(y_pred)
        y_true_all.append(y_true)
    
    y_pred_all = np.array(y_pred_all).flatten()
    y_true_all = np.array(y_true_all).flatten()

    df = pd.DataFrame({
        'Pred': y_pred_all,
        'True': y_true_all
    })

    df.to_csv(f'2_RNAmodel_vexter/geneSumscore/Hyperparameters/HyperCombination_{idx}.csv', index=False)

2. Hyperparameter evaluation

In [37]:
import pandas as pd
import glob
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

dfList = glob.glob('HyperCombination_*.csv')
accAll, f1All = [],[]
for dfPath in dfList:
    df = pd.read_csv(dfPath)
    y_pred = df['Pred']
    y_true = df['True']

    accAll.append(accuracy_score(y_true, y_pred))
    f1All.append(f1_score(y_true, y_pred, average='weighted'))
    
mtrx = pd.DataFrame({'idx':range(16),
            'acc': accAll,
            'f1': f1All
            })

new_mtrx = mtrx.sort_values(by='acc', ascending=False)
print(new_mtrx.iloc[0]) 
top_idx = new_mtrx.iloc[0,0]  

idx    5.000000
acc    0.703077
f1     0.706936
Name: 5, dtype: float64


In [38]:
print(param_combinations[top_idx])
# param= {'max_depth':1,
#         'n_estimators': 30,
#         'learning_rate': 0.3}

(1, 30, 0.3)


# Evaluation

## calulate matics

In [20]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [21]:
cohort = 'exterTest' # exterTest
dfPath = f'evaluation/evaluation_{cohort}.csv'  
df = pd.read_csv(dfPath)
print(df.shape)

# 2 classes
df2 = df.loc[(df['Pred']!= 2)& (df['True']!= 2)]  

accList, preList, recallList, f1List = [],[],[],[]
for idx in range(10):

    df_plot = df2[df2['Seed'] == idx]
    y_true = df_plot['True']
    y_pred = df_plot['Pred']
    y_prob = df_plot['HCCprob']

    accList.append(accuracy_score(y_true, y_pred))
    preList.append(precision_score(y_true, y_pred))
    recallList.append(recall_score(y_true, y_pred))
    f1List.append(f1_score(y_true, y_pred))

mtrx = pd.DataFrame({
    'acc': accList,
    'precision':preList,
    'recall':recallList,
    'f1':f1List
})
mtrx.to_csv(f'evaluation/mtrx_{cohort}.csv')

(1050, 4)


In [ ]:
print(f'Accuracy is {mtrx["acc"].mean(),mtrx["acc"].std()}')
print(f'Precision is {mtrx["precision"].mean(),mtrx["precision"].std()}')
print(f'Recall is {mtrx["recall"].mean(),mtrx["recall"].std()}')
print(f'F1 is {mtrx["f1"].mean(),mtrx["f1"].std()}')

In [ ]:
# external test set：
dfPath = f'evaluation/evaluation_exterTest.csv' 
df = pd.read_csv(dfPath)
print(df.shape)

df_plot = df.loc[(df['Pred']!= 2)& (df['True']!= 2)]  # 注意2是Y

In [ ]:
y_true = df_plot['True']
y_pred = df_plot['Pred']
y_prob = df_plot['HCCprob']

print(f'Accuracy is {accuracy_score(y_true, y_pred)}')
print(f'Precision is {precision_score(y_true, y_pred)}')
print(f'Recall is {recall_score(y_true, y_pred)}')
print(f'F1 is {f1_score(y_true, y_pred)}')
print(f'AUC is {roc_auc_score(y_true, y_prob)}')
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix= cm)
disp.plot()
plt.show()

# Visualization

## ROC 
DN & HCC

In [1]:
import seaborn as sns
from sklearn.metrics import auc, roc_curve, RocCurveDisplay
import matplotlib.pyplot as plt
import pickle
import numpy as np
import os
import pandas as pd


In [17]:
dfPath = 'geneSumscore/evaluation/evaluation_exterTest.csv' # inter_5fold, 
df = pd.read_csv(dfPath)
print(df.shape)

df_plot = df.loc[(df['Pred']!= 2)& (df['True']!= 2)]

fpr, tpr, _ = roc_curve(df_plot['True'], df_plot['HCCprob'])
roc_auc = auc(fpr, tpr)

(21, 4)


In [15]:
cohort = 'exterTest'   # inter_5fold, exterTest
dfPath = f'geneSumscore/evaluation/evaluation_{cohort}.csv' # inter_5fold, exterTest
df = pd.read_csv(dfPath)
print(df.shape)

roc_curves = []
auc_values = []

for i in range(10): # load results
    df_seed = df.loc[df['Seed'] == i,:]
    df_plot = df_seed.loc[(df['Pred']!= 2)& (df['True']!= 2)]
    
    fpr, tpr, _ = roc_curve(df_plot['True'], df_plot['HCCprob'])
    roc_curves.append((fpr, tpr))
    auc_values.append(auc(fpr, tpr))

(1050, 4)


In [ ]:
mean_fpr = np.linspace(0, 1, 100)
mean_tpr = np.mean([np.interp(mean_fpr, curve[0], curve[1]) for curve in roc_curves], axis=0)
mean_auc = np.mean(auc_values)
std_auc = np.std(auc_values)
print(mean_auc, std_auc)

std_tpr = np.std([np.interp(mean_fpr, curve[0], curve[1]) for curve in roc_curves], axis=0)

# draw ROC
plt.figure(figsize=(3, 3))
plt.plot(mean_fpr, mean_tpr, color='red', label='AUROC = {:.2f}'.format(mean_auc))
plt.fill_between(mean_fpr, mean_tpr - std_tpr, mean_tpr + std_tpr, color='#FF7F7F', alpha=0.5)

# draw others
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.grid()
plt.savefig(f'geneSumscore/evaluation/ROC_{cohort}.pdf')
plt.show()

## confusion matrix

In [17]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import pickle

In [18]:
dfPath = 'geneSumscore/evaluation/evaluation_exterTest.csv' # exterTest, inter_5fold
df = pd.read_csv(dfPath)
print(df.shape)

# 2 classes
df2 = df.loc[(df['Pred']!= 2)& (df['True']!= 2)]
y_true = df2['True']
y_pred = df2['Pred']

(1050, 4)


In [ ]:
cohort = 'External institute test'  # External institute test, Validation
label_mapping = {0: 'DN', 1: 'HCC'}
cm_outpath = f'geneSumscore/evaluation/cm_2{cohort}.pdf'

R_CMAP = sns.light_palette('#D80000', as_cmap=True, n_colors=256)

target_types = np.sort(pd.unique(y_true))
cm = confusion_matrix(y_true, y_pred)

cm_percentage = np.round(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis],3)
cm_df = pd.DataFrame(cm_percentage, index=target_types, columns=target_types)
cm_df.columns = cm_df.columns.map(label_mapping)
cm_df.index = cm_df.index.map(label_mapping)


plt.figure(figsize=(4, 3))  # Set figure size for better visibility
ax = sns.heatmap(cm_df, annot=True, fmt='.3f', square=True, 
                linewidths=2, linecolor='white', cmap=R_CMAP,  # Example cmap
                cbar=False, xticklabels=True)

# Set labels and title
ax.set_xlabel('Predicted label')
ax.set_ylabel('True label')
ax.set_title(f'{cohort} dataset')

# Save the figure
plt.tight_layout()  # Adjust layout to avoid clipping of labels
plt.savefig(cm_outpath, bbox_inches='tight')
plt.show()  # Optional: Display the plot